In [26]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

In [27]:
from numpy.testing import assert_array_equal

import logging
logging.basicConfig(level=logging.INFO)

In [28]:
import structlib
from structlib.visualize import show_atoms_2d, show_atoms_3d
from structlib.disloc import disloc_loop_z, disloc_poly_z, disloc_edge

In [29]:
cell_prim = structlib.make.wurtzite('AlN', 3.13, 5.02, cell='prim')
cell_ortho = structlib.make.wurtzite('AlN', 3.13, 5.02, cell='ortho')

show_atoms_3d(cell_prim);

In [30]:
from structlib.cell import plane_to_zone, reduce_vec

assert_array_equal(plane_to_zone(cell_prim.metric, [1., 0., 0.]), [2., 1., 0.])
assert_array_equal(reduce_vec([4., 2., 0.]), [2, 1, 0])

In [31]:
# make supercell

cell = cell_ortho.repeat((8, 5, 5)).explode()
show_atoms_3d(cell);

In [32]:
# add edge dislocation to supercell

edge = disloc_edge(cell, center=[0., 12., 12.], b=[0., 0., 5.02/2.], t=[1., 0., 0.], cut='add')
show_atoms_3d(edge);

In [25]:
# add vacancies to supercell

vacant = cell.with_occupancy(1 - 1000. / len(cell)).apply_occupancy()  # average 1000 vacancies
print(f"Removed {len(cell) - len(vacant)} atoms")

vacant.write('vacancies2.xsf')

In [33]:
from structlib.transform import AffineTransform3D
import polars

faults = cell.transform_atoms(AffineTransform3D.translate(0., 5.421/3, 0.), (polars.col('z') > 11.) & (polars.col('z') < 26.), frame='local')
faults = faults.wrap()
show_atoms_2d(faults.crop_atoms(x_min=16., x_max=30.), zone=[1,0,0]);

faults.write('fault_pair.xsf')

In [52]:
# weird hourglass dislocation shape

poly = [[-8., -8.], [8., -8.], [-8., 8.], [8., 8.]]
loop = disloc_poly_z(cell, [0., 5.421/3, 5.02/2], poly, center=cell.box_size / 2.)

show_atoms_3d(loop.crop_atoms(x_min=5., x_max=20.));

In [53]:
# circular extrinsic dislocation loop
loop = disloc_loop_z(cell, center=cell.box_size / 2., b=[0., 5.421/3, 5.02/2.,], loop_r=8.)

show_atoms_3d(loop);